In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import math
import pandas as pd
import numpy as np

In [2]:
#Inicializa o webdriver e abre o site do Conselho Administrativo de Recursos Fiscais (CARF)

options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(executable_path=r"C:\users\liber\chromedriver.exe", options=options)
driver.get("http://idg.carf.fazenda.gov.br/jurisprudencia/acordaos-carf-2")
driver.find_element_by_xpath('//body/div[2]/div[2]/div/div/div[3]/div/div/div/div/div[2]/div/h3[3]/a').click()

In [3]:
#(i) Pede ao usuário os parâmetros da pesquisa;
#(ii) Faz a pesquisa e informa a quantidade de acórdãos recuperados

elem_mes_ini = driver.find_element_by_xpath(
    '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div/span[2]/input')
elem_mes_fin = driver.find_element_by_xpath(
    '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div/span[4]/input')
elem_pesq_ementa = driver.find_element_by_xpath(
    '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div[4]/input')
botao_pesq = driver.find_element_by_xpath('//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div[5]/input')

elem_mes_ini.clear()
elem_mes_fin.clear()
mes_ini = input('Entre com o mês inicial para pesquisa (MM/AAAA): ')
mes_fin = input('Entre com o mês final para pesquisa (MM/AAAA): ')
t_pesq = input('Entre com o termo de pesquisa: ')
elem_mes_ini.send_keys(mes_ini)
elem_mes_fin.send_keys(mes_fin)
elem_pesq_ementa.send_keys(t_pesq)

print()
print("Período pesquisado:", mes_ini, "a", mes_fin)
print("Termo pesquisado:", t_pesq)
botao_pesq.click()
time.sleep(20)

elem_qtde_acordaos = driver.find_element_by_xpath('//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/span[3]')
qtde_acordaos = int(elem_qtde_acordaos.text)

#n representa a quantidade de páginas que deverão ser percorridas
n = math.ceil(qtde_acordaos/10)
#q representa a quantidade de acórdãos existentes na última página
q = qtde_acordaos - (n-1)*10

print(qtde_acordaos, "acórdãos a serem recuperados, em", n, "páginas.")

Entre com o mês inicial para pesquisa (MM/AAAA): 01/2016
Entre com o mês final para pesquisa (MM/AAAA): 12/2016
Entre com o termo de pesquisa: irpj

Período pesquisado: 01/2016 a 12/2016
Termo pesquisado: irpj
1221 acórdãos a serem recuperados, em 123 páginas.


In [8]:
#Recupera os acórdãos e exporta para um arquivo csv, utilizando o pandas

l_colunas = ['nome_cont', 'nro_proc', 'ano_fisc', 'tipo_rec', 'nro_acordao', 'dt_secao', 'relator',
             'decisao', 'ementa']
l_registros = []

for page in range(n):
    if page != 0:
        botao_prox_pag = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((
            By.XPATH, "//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[3]/div/table/tbody/tr/td[5]")))
        botao_prox_pag.click()
        time.sleep(22)
    
    print("Recuperando os acórdãos da " + str(page+1) + "ª página...")
    
    #Se estiver na última página (n-1), há apenas q acórdãos na página. Senão, há 10 acórdãos por página
    rep = 10
    if page == n-1:
        rep = q
    
    for i in range(rep):
        if i == 0:
            link_acordao = driver.find_element_by_xpath(
                '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/table/tbody/tr/td/div/div/a')
            link_acordao.click()
        else:
            link_acordao = driver.find_element_by_xpath(
                '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/table/tbody/tr['+str(i+1)+']/td/div/div/a')
            link_acordao.click()
        
        #Neste ponto, estamos na página onde se quer fazer a raspagem dos dados
        elem_nro_proc = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr/td/a')
        nro_proc = elem_nro_proc.text
        ano_fisc = nro_proc[-7:-3]
        
        elem_nome_cont = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[2]/td/span[2]')
        nome_cont = elem_nome_cont.text

        elem_tipo_rec = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[3]/td/div/span[2]')
        tipo_rec = elem_tipo_rec.text

        elem_dt_secao = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[3]/td[2]/span[2]')
        dt_secao = elem_dt_secao.text

        elem_relator = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[4]/td/span[2]')
        relator = elem_relator.text

        elem_nro_acordao = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[5]/td/span[2]')
        nro_acordao = elem_nro_acordao.text

        elem_decisao = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[6]/td/span[2]')
        decisao = elem_decisao.text

        elem_ementa = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/table/tbody/tr[7]/td/span[2]')
        ementa = elem_ementa.text
        
        #print('Recuperando decisão...')
        t_registro = (nome_cont, nro_proc, ano_fisc, tipo_rec, nro_acordao, dt_secao, relator, decisao, ementa)
        l_registros.append(t_registro)
                
        botao_voltar = driver.find_element_by_xpath(
            '//body/div/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div[3]/input')
        botao_voltar.click()

driver.quit()

df_carf = pd.DataFrame(l_registros, columns = l_colunas)
df_carf.to_csv('decisoes_CARF.csv', index = False)

df_carf.head()

Recuperando os acórdãos da 1ª página...
Recuperando os acórdãos da 2ª página...
Recuperando os acórdãos da 3ª página...
Recuperando os acórdãos da 4ª página...
Recuperando os acórdãos da 5ª página...
Recuperando os acórdãos da 6ª página...
Recuperando os acórdãos da 7ª página...
Recuperando os acórdãos da 8ª página...
Recuperando os acórdãos da 9ª página...
Recuperando os acórdãos da 10ª página...
Recuperando os acórdãos da 11ª página...
Recuperando os acórdãos da 12ª página...
Recuperando os acórdãos da 13ª página...
Recuperando os acórdãos da 14ª página...
Recuperando os acórdãos da 15ª página...
Recuperando os acórdãos da 16ª página...
Recuperando os acórdãos da 17ª página...
Recuperando os acórdãos da 18ª página...
Recuperando os acórdãos da 19ª página...
Recuperando os acórdãos da 20ª página...
Recuperando os acórdãos da 21ª página...
Recuperando os acórdãos da 22ª página...
Recuperando os acórdãos da 23ª página...
Recuperando os acórdãos da 24ª página...
Recuperando os acórdãos d

Recuperando os acórdãos da 199ª página...
Recuperando os acórdãos da 200ª página...
Recuperando os acórdãos da 201ª página...
Recuperando os acórdãos da 202ª página...
Recuperando os acórdãos da 203ª página...
Recuperando os acórdãos da 204ª página...
Recuperando os acórdãos da 205ª página...
Recuperando os acórdãos da 206ª página...
Recuperando os acórdãos da 207ª página...
Recuperando os acórdãos da 208ª página...
Recuperando os acórdãos da 209ª página...
Recuperando os acórdãos da 210ª página...
Recuperando os acórdãos da 211ª página...
Recuperando os acórdãos da 212ª página...
Recuperando os acórdãos da 213ª página...
Recuperando os acórdãos da 214ª página...
Recuperando os acórdãos da 215ª página...
Recuperando os acórdãos da 216ª página...
Recuperando os acórdãos da 217ª página...
Recuperando os acórdãos da 218ª página...
Recuperando os acórdãos da 219ª página...
Recuperando os acórdãos da 220ª página...
Recuperando os acórdãos da 221ª página...
Recuperando os acórdãos da 222ª pá

,nome_cont,nro_proc,ano_fisc,tipo_rec,nro_acordao,dt_secao,relator,decisao,ementa
0,PEIRO - CORRETORA DE SEGUROS S/S LTDA.,13851.901752/2011-81,2011,RECURSO VOLUNTARIO,1002-001.787,05/11/2020,Não informado,"Vistos, relatados e discutidos os presentes au...",ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA JURÍD...
1,PLASTICOS AZENHA LTDA,11080.913211/2010-68,2010,RECURSO VOLUNTARIO,1002-001.837,02/12/2020,Rafael Zedral,"Vistos, relatados e discutidos os presentes au...",ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA JURÍD...
2,PLASTICOS AZENHA LTDA,11080.905184/2010-50,2010,RECURSO VOLUNTARIO,1002-001.836,02/12/2012,Rafael Zedral,"Vistos, relatados e discutidos os presentes au...",ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA JURÍD...
3,RADIO GIRUA LTDA,11070.908420/2009-75,2009,RECURSO VOLUNTARIO,1002-001.790,05/11/2020,Não informado,"Vistos, relatados e discutidos os presentes au...",ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA JURÍD...
4,EBF PRESTADORA DE SERVICOS LTDA,13971.913902/2009-19,2009,RECURSO VOLUNTARIO,1002-001.783,04/11/2020,Não informado,"Vistos, relatados e discutidos os presentes au...",ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA JURÍD...


In [5]:
driver.quit()